In [1]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [2]:
data_good = ["I like machine learning",
        "I love coding in python",
        "I love building chatbots",
        "I dont hate you",
        "they chat amagingly well"]

data_bad = ["I dont like machine learning",
        "I hate coding in python",
        "I really hate building chatbots",
        "they chat really bad"]


In [16]:
import numpy as np
from sklearn.model_selection import train_test_split

#use 1 for positive sentiment, 0 for negative
y = np.concatenate((np.ones(len(data_good)), np.zeros(len(data_bad))))
x_train, x_test, y_train, y_test = train_test_split(np.concatenate((data_good, data_bad)), y, test_size=0.2)

In [17]:
print('x_train:', x_train)
print('y_train:', y_train)
print('x_test:', x_test)
print('y_test:', y_test)

x_train: ['I really hate building chatbots' 'I hate coding in python'
 'I love building chatbots' 'they chat really bad'
 'they chat amagingly well' 'I like machine learning' 'I dont hate you']
y_train: [0. 0. 1. 0. 1. 1. 1.]
x_test: ['I dont like machine learning' 'I love coding in python']
y_test: [0. 1.]


In [18]:
import nltk
from nltk.corpus import stopwords

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [19]:
from gensim.models.doc2vec import TaggedDocument

    
train_tagged = [TaggedDocument(words=tokenize_text(s), tags=[str(int(y_train[idx]))]) for (idx, s) in enumerate(x_train)]
test_tagged = [TaggedDocument(words=tokenize_text(s), tags=[str(int(y_test[idx]))]) for (idx, s) in enumerate(x_test)]

train_tagged

[TaggedDocument(words=['really', 'hate', 'building', 'chatbots'], tags=['0']),
 TaggedDocument(words=['hate', 'coding', 'in', 'python'], tags=['0']),
 TaggedDocument(words=['love', 'building', 'chatbots'], tags=['1']),
 TaggedDocument(words=['they', 'chat', 'really', 'bad'], tags=['0']),
 TaggedDocument(words=['they', 'chat', 'amagingly', 'well'], tags=['1']),
 TaggedDocument(words=['like', 'machine', 'learning'], tags=['1']),
 TaggedDocument(words=['dont', 'hate', 'you'], tags=['1'])]

In [20]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [21]:
from gensim.models import Doc2Vec

model_dbow = Doc2Vec(dm=1, size=100, negative=5, min_count=0, workers=cores, alpha=0.065, min_alpha=0.065)

model_dbow.build_vocab([x for x in train_tagged])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gensim/models/doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [22]:
from sklearn import utils
from tqdm import tqdm

for epoch in range(10):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged)]), total_examples=len(train_tagged), epochs=10)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

model_dbow.save("d2v_2.model")
print("Model Saved")

100%|██████████| 7/7 [00:00<00:00, 43176.66it/s]


Model Saved


In [23]:
from gensim.models.doc2vec import Doc2Vec

model_dbow = Doc2Vec.load("d2v_2.model")
#to find the vector of a document which is not in training data
test_data = tokenize_text("I dont hate machine learning".lower())

print('test data', test_data)

# model.most_similar('learning')

v = model_dbow.infer_vector(test_data)
model_dbow.docvecs.most_similar([v], topn=len(model_dbow.docvecs))


test data ['dont', 'hate', 'machine', 'learning']


[('1', 0.9668535590171814), ('0', 0.9651862978935242)]

In [24]:
def vec_for_learning(model, tagged_docs):
        sents = tagged_docs
        targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
        return targets, regressors

In [28]:
from sklearn.linear_model import SGDClassifier
lr = SGDClassifier(loss='log', penalty='l1')
lr.fit(vec_for_learning(x_train), vec_for_learning(y_train))
print('Test Accuracy: %.2f'%lr.score(x_test, y_test))

TypeError: vec_for_learning() missing 1 required positional argument: 'tagged_docs'

In [ ]:
y_pred = lr.predict(X_test)
test_acc = accuracy_score(y_test, y_pred) * 100.
loss_values = classifier.loss_curve_
print (loss_values)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
plt.plot(loss_values)
plt.show()

In [ ]:
test_data = tokenize_text("I hate chatbots".lower())

print('test data', test_data)

v = model_dbow.infer_vector(test_data)
model_dbow.docvecs.most_similar([v], topn=len(model_dbow.docvecs))


In [ ]:
model_dbow.most_similar('hate')